In [1]:
import pandas as pd
from pandas import read_parquet
nlst = pd.read_parquet('../../processed_data/nlst.parquet')

nlst.head()

,images,report_text,label,metadata
0,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",Patient ID: 100004\nScreening Year: 0\nIsolati...,1,"{'is_rgb': True, 'name': 'NLST1'}"
1,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",Patient ID: 100004\nScreening Year: 1\nIsolati...,1,"{'is_rgb': True, 'name': 'NLST1'}"
2,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",Patient ID: 100005\nScreening Year: 0\nIsolati...,1,"{'is_rgb': True, 'name': 'NLST1'}"
3,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",Patient ID: 100005\nScreening Year: 1\nIsolati...,1,"{'is_rgb': True, 'name': 'NLST1'}"
4,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",Patient ID: 100005\nScreening Year: 2\nIsolati...,1,"{'is_rgb': True, 'name': 'NLST1'}"


In [2]:
value = nlst.iloc[1, 0]
print(value)

b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \'fortran_order\': False, \'shape\': (3, 224, 224), }                                                   \n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x01\x02\x01\x01\x04\x02\x02\x02\x02\x03\x02\x01\x02\x02\x02\x02\x03\x02\x01\x01\x02\x02\x03\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [3]:
import numpy as np
import io

image_data = nlst['images'][0]
buffer = io.BytesIO(image_data)
image_array = np.load(buffer)

image_array

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [4]:
data = nlst['report_text'][0]
data

'Patient ID: 100004\nScreening Year: 0\nIsolation reading of the image gives the preliminary result of positive, change unspecified, nodule(s) >= 4 mm or enlarging nodule(s), mass(es), other non-specific abnormalities suspicious for lung cancer.\nAbnormality #1 of the patient is a(n) Non-calcified nodule or mass (opacity >= 4 mm diameter); the epicenter of the nodule is located in Right Upper Lobe; the longest diameter found is 4mm in CT slice #26, and the longest diameter perpendicular to it is 3mm; the margin of the nodule is Smooth; the predominant attenuation of the nodule is Soft Tissue.\n'

In [5]:
nlst = pd.read_parquet('../../processed_data/nlst.parquet')

nlst['PID'] = nlst['report_text'].str.extract(r'Patient ID:\s*(\d+)')

missing_pid = nlst['PID'].isnull().sum()
if missing_pid > 0:
    print(f"Warning: {missing_pid} rows failed to extract PID")

new_df = nlst[['PID', 'images', 'label', 'report_text']]

new_df =  new_df.rename(columns={'label': 'cvd'})

def classify_cancer(report_text):
    if 'negative' in report_text.lower():
        return 0
    elif 'positive' in report_text.lower():
        return 1
    else:
        return 2

new_df['cancer'] = new_df['report_text'].apply(classify_cancer)

new_df = new_df.drop(columns=['report_text'])

new_df = new_df.sort_values(by='PID').reset_index(drop=True)

new_df.to_csv('../../processed_data/nlst.csv', index=False, encoding='utf-8')

print("CSV file has been successfully saved as 'nlst.csv'")

CSV file has been successfully saved as 'nlst.csv'


In [6]:
new_df.head()

,PID,images,cvd,cancer
0,100002,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",0,0
1,100002,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",0,0
2,100002,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",0,0
3,100002,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",0,0
4,100002,"b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...",0,0


In [7]:
new_df['PID'].min()

'100002'

In [8]:
new_df.shape[0]

17765

In [9]:
cvd_1_count = (new_df['cvd'] == 1).sum()
cvd_0_count = (new_df['cvd'] == 0).sum()
print(f"Number of rows where cvd = 1: {cvd_1_count}")
print(f"Number of rows where cvd = 0: {cvd_0_count}")

Number of rows where cvd = 1: 12285
Number of rows where cvd = 0: 5480


In [10]:
conflicting_pids = new_df.groupby('PID')['cvd'].nunique()
conflicting_pids = conflicting_pids[conflicting_pids > 1]

if not conflicting_pids.empty:
    print("The following PIDs have conflicting CVD values:")
    conflicting_rows = new_df[new_df['PID'].isin(conflicting_pids.index)]
    print(conflicting_pids.index.tolist())
    print(conflicting_rows)
    
    # for pid in conflicting_pids.index:
    #     pid_rows = conflicting_rows[conflicting_rows['PID'] == pid]
    #     if pid_rows['images'].nunique() > 1:
    #         print(f"PID {pid} has conflicting CVD values and different images:")
    #         print(pid_rows)
    #     else:
    #         print(f"PID {pid} has conflicting CVD values but the same images:")
    #         print(pid_rows)
else:
    print("No conflicting CVD values found for any PID.")

The following PIDs have conflicting CVD values:
['100004', '100019', '100040', '100053', '100072', '100079', '100080', '100092', '100108', '100111', '100130', '100136', '100138', '100141', '100158', '100176', '100188', '100194', '100206', '100214', '100217', '100248', '100252', '100262', '100266', '100280', '100282', '100291', '100327', '100334', '100343', '100346', '100373', '100376', '100380', '100382', '100395', '100397', '100398', '100422', '100427', '100430', '100439', '100459', '100463', '100471', '100472', '100482', '100483', '100515', '100517', '100526', '100532', '100539', '100544', '100547']
         PID                                             images  cvd  cancer
30    100004  b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...    0       0
31    100004  b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...    0       0
32    100004  b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...    0       0
33    100004  b'\x93NUMPY\x01\x00v\x00{\'descr\': \'|u1\', \...    0       0
34    

In [11]:
cancer_counts = new_df['cancer'].value_counts()

print("Cancer classification counts:")
print(f"Cancer = 0 (negative): {cancer_counts.get(0, 0)}")
print(f"Cancer = 1 (positive): {cancer_counts.get(1, 0)}")
print(f"Cancer = 2 (unknown):  {cancer_counts.get(2, 0)}")

Cancer classification counts:
Cancer = 0 (negative): 5561
Cancer = 1 (positive): 12204
Cancer = 2 (unknown):  0


In [14]:
def count_pid_rows_and_check(file_path):
    df = pd.read_csv(file_path)
    if 'PID' not in df.columns:
        raise ValueError("The provided CSV file does not contain a 'PID' column.")
    pid_counts = df['PID'].value_counts().to_dict()
    all_thirty = all(count == 30 for count in pid_counts.values())
    return pid_counts, all_thirty

file_path = '../../processed_data/nlst.csv'
pid_row_counts, all_thirty = count_pid_rows_and_check(file_path)

print(pid_row_counts)
if all_thirty:
    print("Every PID has exactly 30 rows.")
else:
    print("Not every PID has exactly 30 rows.")


{100037: 30, 100075: 30, 100009: 30, 100543: 30, 100116: 30, 100121: 30, 100137: 30, 100081: 30, 100088: 30, 100089: 30, 100091: 30, 100093: 30, 100550: 30, 100551: 30, 100555: 30, 100559: 30, 100560: 30, 100506: 30, 100511: 30, 100519: 30, 100524: 30, 100528: 30, 100534: 30, 100535: 30, 100537: 30, 100540: 30, 100153: 30, 100159: 30, 100160: 30, 100164: 30, 100165: 30, 100166: 30, 100173: 30, 100098: 30, 100099: 30, 100102: 30, 100100: 30, 100249: 30, 100185: 30, 100187: 30, 100197: 30, 100200: 30, 100201: 30, 100203: 30, 100142: 30, 100143: 30, 100480: 30, 100490: 30, 100493: 30, 100494: 30, 100501: 30, 100451: 30, 100456: 30, 100512: 30, 100516: 30, 100277: 30, 100215: 30, 100216: 30, 100220: 30, 100221: 30, 100225: 30, 100226: 30, 100231: 30, 100237: 30, 100239: 30, 100461: 30, 100462: 30, 100469: 30, 100470: 30, 100476: 30, 100417: 30, 100418: 30, 100485: 30, 100253: 30, 100254: 30, 100256: 30, 100258: 30, 100260: 30, 100272: 30, 100209: 30, 100212: 30, 100487: 30, 100488: 30, 100